In [2]:
# 7.2.2 - 퓨샷 프롬프팅 (FewShotPromptTemplate)
# 모델이 예시를 검토해 작업을 더 잘 이해하고 출력 품질을 향상시키는 방법

# 필요한 라이브러리 설치
# !pip install langchain==0.2.17 langchain-openai

# =============================================================================
# API 키 설정 (독자용)
# =============================================================================
import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
    if api_key:
        os.environ['OPENAI_API_KEY'] = api_key
        print("API 키가 설정되었습니다!")
else:
    print("기존 환경 변수의 API 키를 사용합니다.")

# =============================================================================
# 라이브러리 import
# =============================================================================
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_openai import ChatOpenAI

# OpenAI Chat 모델 초기화
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

print("=" * 70)
print("7.2.2 퓨샷 프롬프팅 - 유머러스한 AI 어시스턴트")
print("=" * 70)

print("""
원서 코드의 문제점:
1. LLMChain 클래스가 deprecated됨
2. chain.run() 메소드가 deprecated됨

수정 내용:
- LLMChain(llm=chat, prompt=few_shot_prompt_template) 
  → few_shot_prompt_template | chat
- chain.run({"query": "question"}) 
  → chain.invoke({"query": "question"})

핵심 개념 - FewShotPromptTemplate:
- 예시들을 통해 모델이 원하는 응답 스타일 학습
- prefix: 설명과 맥락 제공
- suffix: 사용자 입력과 출력 지시자 포함
""")

# =============================================================================
# 예시 데이터 정의
# =============================================================================
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    },
    {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

print(f"사용할 예시 개수: {len(examples)}")
for i, example in enumerate(examples, 1):
    print(f"{i}. 질문: {example['query']}")
    print(f"   답변: {example['answer']}")

# =============================================================================
# 예시 포맷 정의
# =============================================================================
example_template = """User: {query}
AI: {answer}"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# =============================================================================
# 프리픽스와 서픽스 정의
# =============================================================================
prefix = """The following are excerpts from conversations with an AI assistant. The assistant
is known for its humor and wit, providing entertaining and amusing responses to users'
questions. Here are some examples:

(다음은 AI 어시스턴트와의 대화에서 발췌한 내용이다. 이 어시스턴트는 유머와 재치를 겸비해 
사용자의 질문에 재미있고 유쾌한 답변을 제공하는 것으로 알려져 있다. 다음은 몇 가지 예시다.)"""

suffix = """User: {query}
AI:"""

# =============================================================================
# FewShotPromptTemplate 생성
# =============================================================================
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

print("\n생성된 프롬프트 구조 확인:")
sample_prompt = few_shot_prompt_template.format(query="What's the secret to happiness?")
print(sample_prompt)
print("\n" + "-" * 50)

# =============================================================================
# 최신 방식으로 실행
# =============================================================================
print("\n실행 결과:")

# 최신 방식: RunnableSequence
chain = few_shot_prompt_template | chat

# 테스트 질문들
test_queries = [
    "What's the secret to happiness?",
    "How do I become successful?",
    "What's your favorite food?",
    "Can you solve world peace?",
    "Why is the sky blue?"
]

try:
    for query in test_queries:
        response = chain.invoke({"query": query})
        print(f"사용자: {query}")
        print(f"AI: {response.content}")
        print("-" * 40)
        
except Exception as e:
    print(f"오류 발생: {e}")

# =============================================================================
# 기존 방식과 비교
# =============================================================================
print("\n" + "=" * 70)
print("기존 방식 vs 새로운 방식")
print("=" * 70)

print("""
원서 코드 (Deprecated):
from langchain.chains import LLMChain

chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
response = chain.run({"query": "What's the secret to happiness?"})

수정된 코드 (현재 권장):
chain = few_shot_prompt_template | chat
response = chain.invoke({"query": "What's the secret to happiness?"})
""")

# =============================================================================
# 다른 스타일의 퓨샷 프롬프팅 예제
# =============================================================================
print("\n" + "=" * 70)
print("다른 스타일 예제: 철학적 조언자")
print("=" * 70)

# 철학적 조언자 예시
philosophical_examples = [
    {
        "query": "What should I do when I'm confused?",
        "answer": "As Socrates said, 'The only true wisdom is in knowing you know nothing.' Embrace the confusion—it's the beginning of learning."
    },
    {
        "query": "How do I find my purpose?",
        "answer": "Viktor Frankl taught us that meaning isn't found, but created through our choices and actions. Start with small acts of service."
    }
]

philosophical_prefix = """The following are excerpts from conversations with a wise AI assistant. The assistant
draws wisdom from philosophy and great thinkers to provide thoughtful, inspiring guidance to users.
Here are some examples:"""

philosophical_template = FewShotPromptTemplate(
    examples=philosophical_examples,
    example_prompt=example_prompt,
    prefix=philosophical_prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

philosophical_chain = philosophical_template | chat

philosophical_questions = [
    "How do I deal with failure?",
    "What makes life worth living?",
    "How can I be happier?"
]

try:
    print("철학적 조언자의 답변:")
    for question in philosophical_questions:
        response = philosophical_chain.invoke({"query": question})
        print(f"질문: {question}")
        print(f"조언: {response.content}")
        print("-" * 40)
        
except Exception as e:
    print(f"철학적 조언자 오류: {e}")

# =============================================================================
# 퓨샷 프롬프팅의 핵심 포인트
# =============================================================================
print("\n" + "=" * 70)
print("퓨샷 프롬프팅 핵심 포인트")
print("=" * 70)

print("""
FewShotPromptTemplate의 구성 요소:
1. examples: 입력-출력 예시 쌍들
2. example_prompt: 각 예시를 포맷하는 템플릿
3. prefix: 전체 맥락과 지시사항
4. suffix: 사용자 입력과 출력 형식 지정
5. example_separator: 예시들 간 구분자

효과적인 퓨샷 프롬프팅 팁:
- 예시 품질이 결과를 좌우함 (고품질 예시 선택 중요)
- 일관된 포맷과 스타일 유지
- 다양한 상황을 커버하는 예시 포함
- 너무 많은 예시는 토큰 낭비 (보통 2-5개가 적당)
- 명확하고 구체적인 prefix 작성

실제 활용 분야:
- 브랜드 톤앤매너 일관성 유지
- 특정 스타일의 콘텐츠 생성
- 도메인별 전문 지식 활용
- 창의적 글쓰기 스타일 모방
""")

print("\n7.2.2 퓨샷 프롬프팅 예제 완료!")

기존 환경 변수의 API 키를 사용합니다.
7.2.2 퓨샷 프롬프팅 - 유머러스한 AI 어시스턴트

원서 코드의 문제점:
1. LLMChain 클래스가 deprecated됨
2. chain.run() 메소드가 deprecated됨

수정 내용:
- LLMChain(llm=chat, prompt=few_shot_prompt_template) 
  → few_shot_prompt_template | chat
- chain.run({"query": "question"}) 
  → chain.invoke({"query": "question"})

핵심 개념 - FewShotPromptTemplate:
- 예시들을 통해 모델이 원하는 응답 스타일 학습
- prefix: 설명과 맥락 제공
- suffix: 사용자 입력과 출력 지시자 포함

사용할 예시 개수: 2
1. 질문: What's the weather like?
   답변: It's raining cats and dogs, better bring an umbrella!
2. 질문: How old are you?
   답변: Age is just a number, but I'm timeless.

생성된 프롬프트 구조 확인:
The following are excerpts from conversations with an AI assistant. The assistant
is known for its humor and wit, providing entertaining and amusing responses to users'
questions. Here are some examples:

(다음은 AI 어시스턴트와의 대화에서 발췌한 내용이다. 이 어시스턴트는 유머와 재치를 겸비해 
사용자의 질문에 재미있고 유쾌한 답변을 제공하는 것으로 알려져 있다. 다음은 몇 가지 예시다.)

User: What's the weather like?
AI: It's raining cats and dogs, better bring an u